# Pipeline for the anomaly detection on the SKAB using Isolation Forest

Details regarding the SKAB one can find in the [SKAB repository](https://github.com/waico/SKAB).

## Libraries importing

In [ ]:
# libraries importing
import sys
import warnings

import matplotlib.pyplot as plt
import pandas as pd

sys.path.append("..")
from core.Isolation_Forest import Isolation_Forest
from core.metrics import chp_score
from core.utils import load_skab

warnings.filterwarnings("ignore", category=UserWarning)

## Data

In [ ]:
list_of_df, anomaly_free_df = load_skab()

## Method

### Method initialization

In [ ]:
# hyperparameters selection
random_state = 0
n_jobs = -1
contamination = 0.0005
PARAMS = [random_state, n_jobs, contamination]

In [ ]:
# model defining
model = Isolation_Forest(PARAMS)

### Method fitting and applying

In [ ]:
# inference
predicted_outlier, predicted_cp = [], []
for df in list_of_df:
    X_train = df[:400].drop(["anomaly", "changepoint"], axis=1)

    # classifier fitting
    model.fit(X_train)

    # results predicting
    prediction = (
        pd.Series(
            model.predict(df.drop(["anomaly", "changepoint"], axis=1)) * -1,
            index=df.index,
        )
        .rolling(3)
        .median()
        .fillna(0)
        .replace(-1, 0)
    )

    # predicted outliers saving
    predicted_outlier.append(prediction)

    # predicted CPs saving
    prediction_cp = prediction.rolling(30).max().fillna(0).diff().abs()
    prediction_cp[0] = prediction[0]
    predicted_cp.append(prediction_cp)

### Results visualization

In [ ]:
# [donotremove]
# true outlier indices selection
true_outlier = [df.anomaly for df in list_of_df]

predicted_outlier[1].plot(
    figsize=(12, 3), label="predictions", marker="o", markersize=5
)
true_outlier[1].plot(marker="o", markersize=2)
plt.legend();

In [ ]:
# [donotremove]
# true changepoint indices selection
true_cp = [df.changepoint for df in list_of_df]

predicted_cp[0].plot(
    figsize=(12, 3), label="predictions", marker="o", markersize=5
)
true_cp[0].plot(marker="o", markersize=2)
plt.legend();

## Evaluation (metrics calculation)

### Binary classification (outlier detection) metrics

In [ ]:
# [donotremove]
# binary classification metrics calculation
binary = chp_score(true_outlier, predicted_outlier, metric="binary")

### Changepoint detection metrics

In [ ]:
# [donotremove]
# average detection delay metric calculation
add = chp_score(
    true_cp,
    predicted_cp,
    metric="average_time",
    anomaly_window_destination="righter",
    portion=1,
)

In [ ]:
# [donotremove]
# nab metric calculation
nab = chp_score(
    true_cp,
    predicted_cp,
    metric="nab",
    window_width="30S",
    anomaly_window_destination="center",
)